In [1]:
import pandas as pd 
import numpy as np
from fuzzywuzzy import process
from sklearn.preprocessing import MinMaxScaler

In [2]:
da=pd.read_csv('emobile.csv')
da.head()

,pid,pname,pratings,nop
0,p1254,samsung mobile-g56,8.3,1214
1,p1682,samsung mobile-f56,6.2,821
2,p1852,vivo-1213,4.2,501
3,p1987,samsung mobile-h56,5.4,623
4,p1365,vivo-v1-pro,8.3,1211


In [3]:
db=pd.read_csv('maxmobile.csv')
db.head()

,mpid,mpname,mpratings,mnop
0,mp1254,samsung mobile-j56,7.3,1114
1,mp1682,samsung mobile-v56,5.2,721
2,mp1852,vivo-1213,3.2,401
3,mp1987,samsung mobile-h56,4.4,523
4,mp1365,vivo-v1-pro,7.3,1111


In [4]:
da.drop_duplicates(inplace=True)
db.drop_duplicates(inplace=True)
da.columns=['Product_ID','Product_Name','Rating','Review_Count']
db.columns=['Product_ID','Product_Name','Rating','Review_Count']

In [5]:
def matching(name,choice,l=5):
    results=process.extract(name,choice,limit=l)
    return results[0][0] if results else None

In [6]:
db['Matched_Product'] = db['Product_Name'].apply(lambda x: matching(x, da['Product_Name'].tolist()))


In [7]:
db.head()

,Product_ID,Product_Name,Rating,Review_Count,Matched_Product
0,mp1254,samsung mobile-j56,7.3,1114,samsung mobile-g56
1,mp1682,samsung mobile-v56,5.2,721,samsung mobile-g56
2,mp1852,vivo-1213,3.2,401,vivo-1213
3,mp1987,samsung mobile-h56,4.4,523,samsung mobile-h56
4,mp1365,vivo-v1-pro,7.3,1111,vivo-v1-pro


In [8]:
merged=pd.merge(da,db, left_on='Product_Name', right_on='Matched_Product', how='inner', suffixes=('_ds1', '_ds2'))
merged.head()

,Product_ID_ds1,Product_Name_ds1,Rating_ds1,Review_Count_ds1,Product_ID_ds2,Product_Name_ds2,Rating_ds2,Review_Count_ds2,Matched_Product
0,p1254,samsung mobile-g56,8.3,1214,mp1254,samsung mobile-j56,7.3,1114,samsung mobile-g56
1,p1254,samsung mobile-g56,8.3,1214,mp1682,samsung mobile-v56,5.2,721,samsung mobile-g56
2,p1852,vivo-1213,4.2,501,mp1852,vivo-1213,3.2,401,vivo-1213
3,p1987,samsung mobile-h56,5.4,623,mp1987,samsung mobile-h56,4.4,523,samsung mobile-h56
4,p1365,vivo-v1-pro,8.3,1211,mp1365,vivo-v1-pro,7.3,1111,vivo-v1-pro


In [9]:
merged.columns

Index(['Product_ID_ds1', 'Product_Name_ds1', 'Rating_ds1', 'Review_Count_ds1',
       'Product_ID_ds2', 'Product_Name_ds2', 'Rating_ds2', 'Review_Count_ds2',
       'Matched_Product'],
      dtype='object')

In [10]:
x=merged['Review_Count_ds1']+merged['Review_Count_ds2']

In [11]:
merged['Combined_Rating']=(merged['Rating_ds1']*merged['Review_Count_ds1']+
                           merged['Rating_ds2']*merged['Review_Count_ds2'])/x
merged.head()

,Product_ID_ds1,Product_Name_ds1,Rating_ds1,Review_Count_ds1,Product_ID_ds2,Product_Name_ds2,Rating_ds2,Review_Count_ds2,Matched_Product,Combined_Rating
0,p1254,samsung mobile-g56,8.3,1214,mp1254,samsung mobile-j56,7.3,1114,samsung mobile-g56,7.821478
1,p1254,samsung mobile-g56,8.3,1214,mp1682,samsung mobile-v56,5.2,721,samsung mobile-g56,7.144910
2,p1852,vivo-1213,4.2,501,mp1852,vivo-1213,3.2,401,vivo-1213,3.755432
3,p1987,samsung mobile-h56,5.4,623,mp1987,samsung mobile-h56,4.4,523,samsung mobile-h56,4.943630
4,p1365,vivo-v1-pro,8.3,1211,mp1365,vivo-v1-pro,7.3,1111,vivo-v1-pro,7.821533


In [12]:
merged['Recommendation_Score']=0.7*merged['Combined_Rating']+0.3*merged['Rating_ds1']
merged.head()

,Product_ID_ds1,Product_Name_ds1,Rating_ds1,Review_Count_ds1,Product_ID_ds2,Product_Name_ds2,Rating_ds2,Review_Count_ds2,Matched_Product,Combined_Rating,Recommendation_Score
0,p1254,samsung mobile-g56,8.3,1214,mp1254,samsung mobile-j56,7.3,1114,samsung mobile-g56,7.821478,7.965034
1,p1254,samsung mobile-g56,8.3,1214,mp1682,samsung mobile-v56,5.2,721,samsung mobile-g56,7.144910,7.491437
2,p1852,vivo-1213,4.2,501,mp1852,vivo-1213,3.2,401,vivo-1213,3.755432,3.888803
3,p1987,samsung mobile-h56,5.4,623,mp1987,samsung mobile-h56,4.4,523,samsung mobile-h56,4.943630,5.080541
4,p1365,vivo-v1-pro,8.3,1211,mp1365,vivo-v1-pro,7.3,1111,vivo-v1-pro,7.821533,7.965073


In [13]:
best=merged.sort_values (by='Recommendation_Score', ascending=False)

In [14]:
print(best[['Matched_Product', 'Combined_Rating', 'Recommendation_Score']].head(5))

   Matched_Product  Combined_Rating  Recommendation_Score
13  one plus x pro         8.707901              8.855531
7      vivo-z1-pro         8.412755              8.558929
10      one plus 2         8.241458              8.349020
9       one plus 2         8.110101              8.257071
5        nokia1100         7.921097              8.064768


In [15]:
best[['Matched_Product', 'Combined_Rating', 'Recommendation_Score']].to_csv('BestRecommended.csv',index=False)


In [16]:
new=pd.read_csv('BestRecommended.csv')
new.head(5)

,Matched_Product,Combined_Rating,Recommendation_Score
0,one plus x pro,8.707901,8.855531
1,vivo-z1-pro,8.412755,8.558929
2,one plus 2,8.241458,8.349020
3,one plus 2,8.110101,8.257071
4,nokia1100,7.921097,8.064768
